In [1]:
import sys
import os

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [2]:
import tensorflow as tf
# tf.compat.v1.enable_eager_execution()

In [3]:
# import torch
# torch.ones((10, 10)).nelement()

In [4]:
import malaya_speech
from malaya_speech.train.model import uis_rnn






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [5]:
observation_dim = 100

model = uis_rnn.Model(observation_dim = observation_dim)

In [6]:
X = tf.placeholder(tf.float32, [None, None, observation_dim])
Y = tf.placeholder(tf.float32, [None, None, observation_dim])

In [7]:
mean, hidden = model(X)
mean, hidden

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


(<tf.Tensor 'uis-rnn/CoreRNN/dense_1/BiasAdd:0' shape=(?, ?, 100) dtype=float32>,
 [<tf.Tensor 'uis-rnn/CoreRNN/lstm/while/Exit_2:0' shape=(?, 512) dtype=float32>,
  <tf.Tensor 'uis-rnn/CoreRNN/lstm/while/Exit_3:0' shape=(?, 512) dtype=float32>])

In [8]:
y_ = tf.transpose(Y, (1, 0, 2))
m = tf.transpose(mean, (1, 0, 2))
m = tf.math.cumsum(m, axis = 0)
mean_size = tf.shape(m)
diag = tf.diag(1.0 / tf.cast(tf.range(1, mean_size[0] + 1), tf.float32))
m = tf.matmul(diag, tf.reshape(m, (mean_size[0], -1)))
m = tf.reshape(m, mean_size)

In [9]:
def sigma2_prior_loss(num_non_zero, sigma_alpha, sigma_beta, sigma2):
    l = tf.reduce_sum((2 * sigma_alpha + num_non_zero + 2) /
          (2 * num_non_zero) * tf.log(sigma2))
    r = tf.reduce_sum(sigma_beta / (sigma2 * num_non_zero))
    return l + r

def weighted_mse_loss(input_tensor, target_tensor, weight=1):
    observation_dim = input_tensor.shape[-1]
    streched_tensor = tf.reshape(((input_tensor - target_tensor) ** 2), (-1, observation_dim))
    entry_num = tf.cast(tf.shape(streched_tensor)[0], tf.float32)
    non_zero_entry_num = tf.reduce_sum(tf.cast(tf.not_equal(streched_tensor[:, 0], 0), tf.float32))
    l = tf.reshape(((input_tensor - target_tensor)**2), (-1, observation_dim))
    r = tf.diag(tf.reshape(weight, (-1,)))
    weighted_tensor = tf.matmul(l, r)
    
    return tf.reduce_mean(weighted_tensor) * tf.cast(weight.shape[0], tf.float32) * entry_num / non_zero_entry_num

In [12]:
input_tensor = tf.cast(tf.math.not_equal(y_, 0), tf.float32) * m[:-1, :, :]
loss1 = weighted_mse_loss(input_tensor, y_, 1.0 / (2.0 * model.sigma2))

In [13]:
weight = tf.reshape((tf.cast(tf.not_equal(y_, 0), tf.float32) * m[:-1, :, :] - y_) ** 2,
           (-1, observation_dim))
num_non_zero = tf.reduce_sum(tf.cast(tf.not_equal(weight, 0), tf.float32), axis = 0)
loss2 = sigma2_prior_loss(num_non_zero, 1.0, 1.0, model.sigma2)

In [14]:
loss = loss1 + loss2
loss

<tf.Tensor 'add_4:0' shape=() dtype=float32>

In [15]:
import numpy as np

train_sequence = np.random.rand(1000, observation_dim)
train_cluster_id = np.array(['A'] * 1000)

In [16]:
train_sequences = [
    np.random.rand(100, 100),
    np.random.rand(200, 100),
    np.random.rand(300, 100)]
train_cluster_ids = [
    np.array(['A'] * 100),
    np.array(['A'] * 200),
    np.array(['A'] * 300),]

In [17]:
uis_rnn.utils.estimate_transition_bias(train_cluster_ids)

(0.001669449081803005, 599)

In [18]:
concatenated_train_sequence, concatenated_train_cluster_id = uis_rnn.utils.concatenate_training_data(
         train_sequences,
         train_cluster_ids,
         True,
         True)

concatenated_train_cluster_id = np.array(concatenated_train_cluster_id)

In [28]:
concatenated_train_sequence.shape

(600, 100)

In [19]:
sub_sequences, seq_lengths = uis_rnn.utils.resize_sequence(
        sequence=concatenated_train_sequence,
        cluster_id=concatenated_train_cluster_id,
        num_permutations=10)

In [20]:
batch_size = 2

packed_train_sequence, rnn_truth = uis_rnn.utils.pack_sequence(
          sub_sequences,
          seq_lengths,
          batch_size,
          observation_dim)

In [29]:
concatenated_train_sequence.shape

(600, 100)

In [21]:
x = np.transpose(packed_train_sequence, (1, 0, 2))
y = np.transpose(rnn_truth, (1, 0, 2))
x.shape, y.shape

((2, 201, 100), (2, 200, 100))

In [22]:
sess = tf.Session()

In [23]:
sess.run(tf.global_variables_initializer())

In [24]:
%%time

sess.run(mean, feed_dict = {X: x, Y: y}).shape

CPU times: user 758 ms, sys: 149 ms, total: 907 ms
Wall time: 219 ms


(2, 201, 100)

In [25]:
%%time

sess.run([loss1, loss2], feed_dict = {X: x, Y: y})

CPU times: user 616 ms, sys: 129 ms, total: 745 ms
Wall time: 194 ms


[172.0286, -113.78055]

In [26]:
%%time

sess.run([loss1, loss2, loss], feed_dict = {X: x, Y: y})

CPU times: user 761 ms, sys: 155 ms, total: 916 ms
Wall time: 233 ms


[172.0286, -113.78055, 58.248047]